In [1]:
import os
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet.gluon import nn
from gluoncv.utils import viz
from mxnet import nd

'''
ConvBlock
[Conv2d ─ BN ─ ACTIVATION]
'''
def ConvBlock(channels, kernel_size, strides, padding, use_bias=False, activation='leaky'):
    block = nn.HybridSequential()
    block.add(nn.Conv2D(int(channels), kernel_size=kernel_size, strides=strides, padding=padding, use_bias=use_bias))

    if not use_bias:
        block.add(nn.BatchNorm(in_channels=int(channels)))

    if activation == 'leaky':
        block.add(nn.LeakyReLU(0.1))

    return block


'''
RedidualBlock
[   ┌───────────────────────┐    ]
[In ┴ Conv(1x1) ─ Conv(3x3) ┴ out]
'''
class ResidualBlock(nn.HybridBlock):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvBlock(channels, kernel_size=1, strides=1, padding=0)
        self.conv2 = ConvBlock(channels, kernel_size=3, strides=1, padding=1)

    def hybrid_forward(self, F, x):
        # print('F: ', F)
        # print('x: ', x.shape, type(x))

        block = self.conv1(x)
        block = self.conv2(block)
        out = block + x

        return out
    
class CSP(nn.HybridBlock):
    def __init__(self, channels, block_size=1):
        super(CSP, self).__init__()
        self.conv0 = ConvBlock(channels, kernel_size=3, strides=2, padding=1)
        self.conv1 = ConvBlock(channels/2, kernel_size=1, strides=1, padding=0)
        self.resblocks = self.make_residual_blocks(channels/2, block_size)
        
        self.conv2 = ConvBlock(channels/2, kernel_size=1, strides=1, padding=0)
        self.conv3 = ConvBlock(channels/2, kernel_size=1, strides=1, padding=0)
        self.conv4 = ConvBlock(channels, kernel_size=1, strides=1, padding=0)
        
    def hybrid_forward(self, F, x):
        x = self.conv0(x)
        short_cut = x        
        x = self.conv1(x)
        x = self.resblocks(x)
        x = self.conv2(x)
        short_cut = self.conv3(short_cut)
        x = F.concat(x, short_cut, dim=1)
        x = self.conv4(x)
        
        return x
        
    def make_residual_blocks(self, channels, block_size):
        layer = nn.HybridSequential()
        for i in range(block_size):
            layer.add(ResidualBlock(channels))        
        return layer
    
class DarkNet(nn.HybridBlock):
    def __init__(self, num_classes=1000, input_size=416):
        super(DarkNet, self).__init__()
        self.layer_num = 0
        self.num_classes = num_classes
        self.input_size = input_size

        self.input_layer = nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1, use_bias=False)

        self.layer1 = CSP(64, 1)
        self.layer2 = CSP(128, 2)
        self.layer3 = CSP(256, 8)
        self.layer4 = CSP(512, 8)
        self.layer5 = CSP(1024, 4)

        self.global_avg_pool = nn.GlobalAvgPool2D()
        self.fc = nn.Dense(self.num_classes)

    def hybrid_forward(self, F, x):
        x = self.input_layer(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.global_avg_pool(x)
        x = self.fc(x)
        
        return x

In [10]:
net = DarkNet(num_classes=5, input_size=224)
net.hybridize()
net.initialize()
x = nd.random.normal(shape=(1, 3, 224, 224))
y = net(x)
net.export("darknet53")
print('y: ', y.shape, type(y))

y:  (1, 5) <class 'mxnet.ndarray.ndarray.NDArray'>


In [3]:
import mxnet as mx
import numpy as np
from mxnet.contrib import onnx as onnx_mxnet

sym = './darknet53-symbol.json'
params = './darknet53-0000.params'

# Standard Imagenet input - 3 channels, 224*224
input_shape = (1,3,224,224)

# Path of the output file
onnx_file = './mxnet_exported_darknet53.onnx'

In [4]:
converted_model_path = onnx_mxnet.export_model(sym, params, [input_shape], np.float32, onnx_file)

In [5]:
import onnxruntime

In [13]:
ort_session = onnxruntime.InferenceSession("mxnet_exported_darknet53.onnx")

x_1 = x.asnumpy()

# ONNX 런타임에서 계산된 결과값
ort_inputs = {ort_session.get_inputs()[0].name: x_1}
ort_outs = ort_session.run(None, ort_inputs)

# ONNX 런타임과 PyTorch에서 연산된 결과값 비교
np.testing.assert_allclose(y.asnumpy(), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 5 / 5 (100%)
Max absolute difference: 0.00081348
Max relative difference: 3.2020512
 x: array([[ 2.133103e-04, -5.835027e-05,  4.961502e-04, -6.672136e-04,
        -1.075906e-03]], dtype=float32)
 y: array([[ 0.001027, -0.000461,  0.000998, -0.000159, -0.001   ]],
      dtype=float32)